0. preprocessing
1. PCA
2. KMC
- <- tried exploring up to this point
3. Improving KMC Performance: cross validation, elbow
4. apply preprocessing before KMC <- this also done but just in a type-changing and default-normalizing way 
5. comparison with DBSCAN, SOM, RNN

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

# exploratory analysis 
import ydata_profiling as yp

# sklearn for clustering algorithms
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

pd.options.display.max_columns = None
sns.set_context("talk")
sns.set_style("whitegrid")

In [2]:
df_sku = pd.read_csv('../data/sku_noid.csv',
                 names=[
    # 0407 added expire date column that was dropped before
                    "unit_price", "Expire date", "outbound_number",
                    "total_outbound", "pal_weight", "pal_height", "units_per_pallet"
                 ], skiprows=1, index_col=False)

df_super = pd.read_csv("../data/superstore.csv")

In [7]:
df_super.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [8]:
# Superstore dataset

# Type Change: obj -> str
# 2 columns
df_super['Customer ID'] = df_super['Customer ID'].astype("string")
# df_super['City'] = df_super['City'].astype("string")
# df_super['State'] = df_super['State'].astype("string")
# df_super['Postal Code'] = df_super['Postal Code'].astype("string")
df_super['Product ID'] = df_super['Product ID'].astype("string")
# df_super['Product Name'] = df_super['Product Name'].astype("string")
# df_super['Sub-Category'] = df_super['Sub-Category'].astype("string")

# Type Change: obj -> category 
# 5 columns
df_super['Ship Mode'] = df_super['Ship Mode'].astype("category")
df_super['Segment'] = df_super['Segment'].astype("category")
df_super['Region'] = df_super['Region'].astype("category")
df_super['Category'] = df_super['Category'].astype("category")
df_super['Sub-Category'] = df_super['Sub-Category'].astype("category")

# Type Change: obj -> datetime
df_super['Order Date'] = pd.to_datetime(df_super['Order Date'])
df_super['Ship Date'] = pd.to_datetime(df_super['Ship Date'])

# Drop Columns:
# columns with no value: Order ID, Customer Name, Country
# 7 columns
df_super_f = df_super.loc[:, ~df_super.columns.isin(
    ['Row ID', 'Order ID', 'Customer Name', 'Product Name', 
     'Country', 'City', 'State', 'Postal Code'])]

In [9]:
df_super_f.head()

,Order Date,Ship Date,Ship Mode,Customer ID,Segment,Region,Product ID,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,South,FUR-BO-10001798,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,South,FUR-CH-10000454,Furniture,Chairs,731.9400,3,0.00,219.5820
2,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,West,OFF-LA-10000240,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,South,FUR-TA-10000577,Furniture,Tables,957.5775,5,0.45,-383.0310
4,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,South,OFF-ST-10000760,Office Supplies,Storage,22.3680,2,0.20,2.5164


## checking relationships for non-categorical string columns
- columns with repetitive meaning may have to be dropped
1. Customer-specific = "Customer ID",
2. Location-specific = "City", "State", "Postal Code"
3. SKU-specific = "Product ID", "Product Name",
4. Should go into categorical feature but will introduce much columns after encoding
- "Sub-Category" #since it has 17 categorical values

## how to encode string columns?
1. LabelEncoder(categorical feature required)

In [ ]:
# datetime to int
df_super_f['Order Date'] = df_super_f['Order Date'].astype(int) / 10**14
df_super_f['Ship Date'] = df_super_f['Ship Date'].astype(int) / 10**14

le_super = LabelEncoder()
string_columns = [
    "Customer ID",
    # "City", 
    # "State",
    # "Postal Code",
    "Product ID",
    # "Product Name",
    # "Sub-Category" #since it has 17 categorical values
]
for i in string_columns:
    df_super_f[i] = le_super.fit_transform(df_super_f[i])

# encode category datatype
category_columns = [
    "Ship Mode", #4
    "Segment", #3
    "Region", #4
    "Category", #3
    "Sub-Category" #17
]

for i in category_columns:
    ohe_super_f = OneHotEncoder()
    ohe_df_super_f = pd.DataFrame(
        ohe_super_f.fit_transform(
            df_super_f[[i]]
        ).toarray())
    # add suffix to column names in ohe_df
    ohe_df_super_f = ohe_df_super_f.add_suffix(i)  
    df_super_f = df_super_f.join(ohe_df_super_f)

In [ ]:
# dropping the category columns and scaling the data
scaler = StandardScaler()
scaled_df_super_f = scaler.fit_transform(
    df_super_f.drop(category_columns, axis=1))

In [5]:
# enc1 = (df_super.groupby('Product Name').size()) / len(df_sku)
# enc1

Product Name
"While you Were Out" Message Book, One Form pe...    0.001316
#10 Gummed Flap White Envelopes, 100/Box             0.001755
#10 Self-Seal White Envelopes                        0.001755
#10 White Business Envelopes,4 1/8 x 9 1/2           0.003072
#10- 4 1/8" x 9 1/2" Recycled Envelopes              0.004388
                                                       ...   
iKross Bluetooth Portable Keyboard + Cell Phon...    0.002194
iOttie HLCRIO102 Car Mount                           0.002194
iOttie XL Car Mount                                  0.000878
invisibleSHIELD by ZAGG Smudge-Free Screen Pro...    0.003072
netTALK DUO VoIP Telephone Service                   0.003072
Length: 1850, dtype: float64

In [ ]:
# Frequency Encoding rather than label encoding
df_super['Product Name_enc'] = df_super['Product Name'].apply(lambda x : enc1[x])